In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()

# 認証情報
auth_url = os.getenv('AICORE_AUTH_URL').strip(' "\',')
client_id = os.getenv('AICORE_CLIENT_ID').strip(' "\',')
client_secret = os.getenv('AICORE_CLIENT_SECRET').strip(' "\',')
base_url = os.getenv('AICORE_BASE_URL').strip(' "\',')
resource_group = os.getenv('AICORE_RESOURCE_GROUP').strip(' "\',')

# トークンを取得する関数
def get_token(auth_url, client_id, client_secret):
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    response = requests.post(auth_url + '/oauth/token', data=data)
    response.raise_for_status()
    return response.json()['access_token']

# トークンを取得
token = get_token(auth_url, client_id, client_secret)

# API にアクセスする例
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json',
    'AI-Resource-Group': resource_group
}

# /v2/lm/scenarios エンドポイントに対してリクエストを送信
response = requests.get(base_url + '/v2/lm/scenarios', headers=headers)

# レスポンスのステータスコードと内容を表示
print(f'Status Code: {response.status_code}')
print(f'Response Text: {response.text}')

In [ ]:
# 設定の作成リクエストデータ
configuration_data = {
    "name": "embedding-itomond",
    "executableId": "azure-openai",
    "scenarioId": "foundation-models",
    "versionId": "0.0.1",
    "parameterBindings": [
        {"key": "modelName", "value": "text-embedding-ada-002"},
        {"key": "modelVersion", "value": "latest"}
    ],
    "inputArtifactBindings": []
}

# 設定の作成リクエスト
response_config = requests.post(base_url + '/v2/lm/configurations', headers=headers, data=json.dumps(configuration_data))

# エラーの詳細を表示
try:
    response_config.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f'HTTP error occurred: {err}')  # HTTP エラーメッセージ
    print(f'Response content: {response_config.content}')  # レスポンスの内容

# レスポンスの表示
configuration = response_config.json()
print(configuration)

In [ ]:
import time

# デプロイメントの作成リクエストデータ
deployment_data = {
    "configurationId": configuration['id']
}

# デプロイメントの作成リクエスト
response_deployment = requests.post(base_url + '/v2/lm/deployments', headers=headers, data=json.dumps(deployment_data))

# エラーの詳細を表示
try:
    response_deployment.raise_for_status()
except requests.exceptions.HTTPError as err:
    print(f'HTTP error occurred: {err}')  # HTTP エラーメッセージ
    print(f'Response content: {response_deployment.content}')  # レスポンスの内容

# レスポンスの表示
deployment = response_deployment.json()
print(deployment)


In [ ]:
# デプロイメントの詳細取得リクエスト
deployment_id = deployment['id']  # 取得したデプロイメントIDを使用

# デプロイメントのステータスが更新されるのを待つための関数
def check_deployment_status(deployment_id):
    status = 'PENDING'
    while status == 'PENDING' or status == 'UNKNOWN':
        response_deployment_status = requests.get(base_url + f'/v2/lm/deployments/{deployment_id}', headers=headers)

        # エラーの詳細を表示
        try:
            response_deployment_status.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(f'HTTP error occurred: {err}')  # HTTP エラーメッセージ
            print(f'Response content: {response_deployment_status.content}')  # レスポンスの内容
            return None

        deployment_status = response_deployment_status.json()
        status = deployment_status.get('status', 'PENDING')

        print(f"Deployment Status: {status}")

        # ステータスが更新されるまで待機
        time.sleep(10)

    return deployment_status

# デプロイメントのステータスをチェック
deployment_status = check_deployment_status(deployment_id)
print(deployment_status)